In [2]:
import logging
from pathlib import Path

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10, FashionMNIST
from torchvision.transforms import v2

from hw2.util import find_mean_and_stddev

In [3]:
logger = logging.getLogger(__name__)

In [4]:
PROJECT_ROOT = Path("../").resolve()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])

In [6]:
cifar10_train_dataset = CIFAR10(root=PROJECT_ROOT / "data", train=True, download=True, transform=transform)
fashion_mnist_train_dataset = FashionMNIST(root=PROJECT_ROOT / "data", train=True, download=True, transform=transform)

Files already downloaded and verified


In [6]:
find_mean_and_stddev(cifar10_train_dataset)

((0.49140113592147827, 0.48215916752815247, 0.44653093814849854),
 (0.24702699482440948, 0.24348318576812744, 0.2615877687931061))

In [7]:
find_mean_and_stddev(fashion_mnist_train_dataset)  # Used as open set data, so this information is not used.

((0.2860410809516907,), (0.3530237376689911,))

In [7]:
transform = v2.Compose([
    v2.ToImage(),
    v2.RGB(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize((0.491, 0.482, 0.446), (0.247, 0.234, 0.262)),
])

In [9]:
cifar10_train_dataset = CIFAR10(root=PROJECT_ROOT / "data", train=True, download=True, transform=transform)
cifar10_test_dataset = CIFAR10(root=PROJECT_ROOT / "data", train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [10]:
fashion_mnist_train_dataset = FashionMNIST(root=PROJECT_ROOT / "data", train=True, download=True, transform=transform)
fashion_mnist_test_dataset = FashionMNIST(root=PROJECT_ROOT / "data", train=False, download=True, transform=transform)

In [11]:
cifar10_train_dataset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: D:\Code\CISC3027\hw2\data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToImage()
                 RGB()
                 ToDtype(scale=True)
                 Normalize(mean=[0.491, 0.482, 0.446], std=[0.247, 0.234, 0.262], inplace=False)
           )

In [12]:
cifar10_train_dataset[0][0].shape

torch.Size([3, 32, 32])

In [13]:
fashion_mnist_train_dataset

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: D:\Code\CISC3027\hw2\data
    Split: Train
    StandardTransform
Transform: Compose(
                 ToImage()
                 RGB()
                 ToDtype(scale=True)
                 Normalize(mean=[0.491, 0.482, 0.446], std=[0.247, 0.234, 0.262], inplace=False)
           )

In [14]:
fashion_mnist_train_dataset[0][0].shape

torch.Size([3, 28, 28])

In [16]:
from hw2.models.cnn_basic import LeNet5

model = LeNet5(channels=3).to(device=device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
train_loader = DataLoader(cifar10_train_dataset, batch_size=4, shuffle=True, num_workers=2)

In [17]:
model.train_loop(criterion, optimizer, train_loader)

[1,   500] loss: 0.531
[1,  1000] loss: 0.483
[1,  1500] loss: 0.445
[1,  2000] loss: 0.424
[1,  2500] loss: 0.423
[1,  3000] loss: 0.404
[1,  3500] loss: 0.396
[1,  4000] loss: 0.399
[1,  4500] loss: 0.395
[1,  5000] loss: 0.370
[1,  5500] loss: 0.392
[1,  6000] loss: 0.375
[1,  6500] loss: 0.381
[1,  7000] loss: 0.364
[1,  7500] loss: 0.358
[1,  8000] loss: 0.349
[1,  8500] loss: 0.353
[1,  9000] loss: 0.349
[1,  9500] loss: 0.358
[1, 10000] loss: 0.357
[1, 10500] loss: 0.353
[1, 11000] loss: 0.349
[1, 11500] loss: 0.347
[1, 12000] loss: 0.348
[1, 12500] loss: 0.351
[2,   500] loss: 0.336
[2,  1000] loss: 0.338
[2,  1500] loss: 0.342
[2,  2000] loss: 0.328
[2,  2500] loss: 0.330
[2,  3000] loss: 0.319
[2,  3500] loss: 0.334
[2,  4000] loss: 0.325
[2,  4500] loss: 0.334
[2,  5000] loss: 0.329
[2,  5500] loss: 0.329
[2,  6000] loss: 0.323
[2,  6500] loss: 0.333
[2,  7000] loss: 0.326
[2,  7500] loss: 0.315
[2,  8000] loss: 0.309
[2,  8500] loss: 0.320
[2,  9000] loss: 0.318
[2,  9500] 

In [19]:
model.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in DataLoader(cifar10_test_dataset, batch_size=4, shuffle=False, num_workers=2):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")

Accuracy of the network on the 10000 test images: 55.28%


In [20]:
# Load my old model
model_hw1 = LeNet5(channels=3)
model_hw1.load_state_dict(torch.load(PROJECT_ROOT / "artifacts" / "202409260908_cnn_Adam_5(60).pth", weights_only=True))
model_hw1.to(device=device)

CNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [21]:
model_hw1.eval()

correct = 0
total = 0

with torch.no_grad():
    for data in DataLoader(cifar10_test_dataset, batch_size=4, shuffle=False, num_workers=2):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model_hw1(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the 10000 test images: {100 * correct / total}%")

Accuracy of the network on the 10000 test images: 52.65%
